# Qiskit ExactReciprocal

This notebook demonstrates the Qiskit `ExactReciprocal` operation.

* [Qiskit ExactReciprocal documentation](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.ExactReciprocal)

The documentation states that the `ExactReciprocal` operation takes the state $|x\rangle|0\rangle$ to:
$$
\cos(\frac{1}{x})|x\rangle|0\rangle + \sin(\frac{1}{x})|x\rangle|1\rangle
$$

However, this is not quite correct. We have found that the operation actually takes the state $|x\rangle|0\rangle$ to:
$$
\sqrt{1-\left(\frac{1}{x}\right)^2}|x\rangle|0\rangle + \frac{1}{x}|x\rangle|1\rangle
$$

As well as the number of qubits for the input value, the operation takes two further parameters:

* `scaling`: Scaling factor $s$ of the reciprocal function, i.e. to compute $s/x$.
* `neg_vals` (Boolean): Whether $x$ might represent negative values. In this case the first qubit is the sign, with $|1\rangle$ for negative and $|0\rangle$ for positive. For the negative case it is assumed that the remaining string represents $1-x$. This is because $e^{-2\pi ix}=e^{2\pi i(1-x)}$ for $x\in[0,1]$.

Jack Morgan's HHL circuit uses `neg_vals=True`.

How does the `ExactReciprocal` operation work? Let's examine some examples.

In [49]:
import numpy as np
from IPython.display import display, Markdown
from math import isclose, log2
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import ExactReciprocal, StatePreparation
from qiskit.circuit.library.generalized_gates import UCRYGate
from qiskit.quantum_info import Statevector


def create_basis_statevector_input_circuit(input):
    x = np.array(input) / np.linalg.norm(input)
    num_state_qubits = int(log2(x.size))
    state_preparation = StatePreparation(Statevector(x))
    x_reg = QuantumRegister(num_state_qubits, 'x')
    circuit = QuantumCircuit(x_reg)
    circuit.append(state_preparation, x_reg)
    return circuit


def create_exact_reciprocal_circuit(input):
    x = np.array(input) / np.linalg.norm(input)
    num_state_qubits = int(log2(x.size))
    scaling = 2**-num_state_qubits
    state_preparation = StatePreparation(Statevector(x))
    x_reg = QuantumRegister(num_state_qubits, 'x')
    a_reg = QuantumRegister(1, 'a')
    circuit = QuantumCircuit(a_reg, x_reg)
    circuit.append(state_preparation, x_reg)
    exact_reciprocal = ExactReciprocal(num_state_qubits=num_state_qubits, scaling=scaling)
    circuit.append(exact_reciprocal, x_reg[::-1] + [a_reg[0]])
    return circuit


def latex_circuit_output(circuit):
    simulator = Aer.get_backend('statevector_simulator')
    aer_circuit = transpile(circuit, simulator)
    return simulator.run(aer_circuit).result().get_statevector().draw(output='latex').data.replace('|', r'\|')


def run_table(vals):
    markdowntable = f"""
| Input $x$ | Input State | Output $\\frac{{s}}{{x}}$ | Output State |
|-----------|-------------|-----------------------|--------------|
"""
    for val in vals:
        input = val["input"]
        input_vec = val["input_vec"]
        output = val["output"]
        input_state = latex_circuit_output(create_basis_statevector_input_circuit(input_vec))
        output_state = latex_circuit_output(create_exact_reciprocal_circuit(input_vec))
        markdowntable += f"| {input} | {input_state} | {output} | {output_state} |\n"
    display(Markdown(markdowntable))


vals2 = [
    {"input": "$0$"           , "input_vec": [1, 0, 0, 0], "output": "$0$"},
    {"input": "$\\frac{1}{2}$", "input_vec": [0, 1, 0, 0], "output": "$\\frac{1}{2}$"},
    {"input": "$\\frac{1}{4}$", "input_vec": [0, 0, 1, 0], "output": "$1$"},
    {"input": "$\\frac{3}{4}$", "input_vec": [0, 0, 0, 1], "output": "$\\frac{1}{3}$"},
]

vals3 = [
    {"input": "$0$"           , "input_vec": [1, 0, 0, 0, 0, 0, 0, 0], "output": "$0$"},
    {"input": "$\\frac{1}{2}$", "input_vec": [0, 1, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{4}$"},
    {"input": "$\\frac{1}{4}$", "input_vec": [0, 0, 1, 0, 0, 0, 0, 0], "output": "$\\frac{1}{2}$"},
    {"input": "$\\frac{3}{4}$", "input_vec": [0, 0, 0, 1, 0, 0, 0, 0], "output": "$\\frac{1}{6}$"},
    {"input": "$\\frac{1}{8}$", "input_vec": [0, 0, 0, 0, 1, 0, 0, 0], "output": "$1$"},
    {"input": "$\\frac{5}{8}$", "input_vec": [0, 0, 0, 0, 0, 1, 0, 0], "output": "$\\frac{1}{5}$"},
    {"input": "$\\frac{3}{8}$", "input_vec": [0, 0, 0, 0, 0, 0, 1, 0], "output": "$\\frac{1}{3}$"},
    {"input": "$\\frac{7}{8}$", "input_vec": [0, 0, 0, 0, 0, 0, 0, 1], "output": "$\\frac{1}{7}$"},
]

vals4 = [
    {"input": "0"               , "input_vec": [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "0"},
    {"input": "$\\frac{1}{2}$"  , "input_vec": [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{8}$"},
    {"input": "$\\frac{1}{4}$"  , "input_vec": [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{4}$"},
    {"input": "$\\frac{3}{4}$"  , "input_vec": [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{12}$"},
    {"input": "$\\frac{1}{8}$"  , "input_vec": [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{2}$"},
    {"input": "$\\frac{5}{8}$"  , "input_vec": [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{10}$"},
    {"input": "$\\frac{3}{8}$"  , "input_vec": [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{6}$"},
    {"input": "$\\frac{7}{8}$"  , "input_vec": [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{14}$"},
    {"input": "$\\frac{1}{16}$" , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], "output": "$1$"},
    {"input": "$\\frac{9}{16}$" , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], "output": "$\\frac{1}{9}$"},
    {"input": "$\\frac{5}{16}$" , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], "output": "$\\frac{1}{5}$"},
    {"input": "$\\frac{13}{16}$", "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], "output": "$\\frac{1}{13}$"},
    {"input": "$\\frac{3}{16}$" , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], "output": "$\\frac{1}{3}$"},
    {"input": "$\\frac{11}{16}$", "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], "output": "$\\frac{1}{11}$"},
    {"input": "$\\frac{7}{16}$" , "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], "output": "$\\frac{1}{7}$"},
    {"input": "$\\frac{15}{16}$", "input_vec": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], "output": "$\\frac{1}{15}$"},
]

input = latex_circuit_output(create_basis_statevector_input_circuit([0, 1, 0, 0, 0, 0, 0, 0]))
output = latex_circuit_output(create_exact_reciprocal_circuit([0, 1, 0, 0, 0, 0, 0, 0]))

markdowntable = f"""
| Input | Output |
|--------------|------------------|
| {input} | {output} |
"""

In [50]:
# num_state_qubits = 2; scaling = 1/4
run_table(vals2)



| Input $x$ | Input State | Output $\frac{s}{x}$ | Output State |
|-----------|-------------|-----------------------|--------------|
| $0$ | $$ \|00\rangle$$ | $0$ | $$ \|000\rangle$$ |
| $\frac{1}{2}$ | $$ \|01\rangle$$ | $\frac{1}{2}$ | $$\frac{\sqrt{3}}{2} \|010\rangle+\frac{1}{2} \|011\rangle$$ |
| $\frac{1}{4}$ | $$ \|10\rangle$$ | $1$ | $$ \|101\rangle$$ |
| $\frac{3}{4}$ | $$ \|11\rangle$$ | $\frac{1}{3}$ | $$\frac{2 \sqrt{2}}{3} \|110\rangle+\frac{1}{3} \|111\rangle$$ |


In [51]:
# num_state_qubits = 3; scaling = 1/8
run_table(vals3)


| Input $x$ | Input State | Output $\frac{s}{x}$ | Output State |
|-----------|-------------|-----------------------|--------------|
| $0$ | $$ \|000\rangle$$ | $0$ | $$ \|0000\rangle$$ |
| $\frac{1}{2}$ | $$ \|001\rangle$$ | $\frac{1}{4}$ | $$\frac{\sqrt{15}}{4} \|0010\rangle+\frac{1}{4} \|0011\rangle$$ |
| $\frac{1}{4}$ | $$ \|010\rangle$$ | $\frac{1}{2}$ | $$\frac{\sqrt{3}}{2} \|0100\rangle+\frac{1}{2} \|0101\rangle$$ |
| $\frac{3}{4}$ | $$ \|011\rangle$$ | $\frac{1}{6}$ | $$0.9860132972 \|0110\rangle+\frac{1}{6} \|0111\rangle$$ |
| $\frac{1}{8}$ | $$ \|100\rangle$$ | $1$ | $$ \|1001\rangle$$ |
| $\frac{5}{8}$ | $$ \|101\rangle$$ | $\frac{1}{5}$ | $$0.9797958971 \|1010\rangle+\frac{1}{5} \|1011\rangle$$ |
| $\frac{3}{8}$ | $$ \|110\rangle$$ | $\frac{1}{3}$ | $$\frac{2 \sqrt{2}}{3} \|1100\rangle+\frac{1}{3} \|1101\rangle$$ |
| $\frac{7}{8}$ | $$ \|111\rangle$$ | $\frac{1}{7}$ | $$0.9897433186 \|1110\rangle+\frac{1}{7} \|1111\rangle$$ |


In [52]:
# num_state_qubits = 4; scaling = 1/16
run_table(vals4)


| Input $x$ | Input State | Output $\frac{s}{x}$ | Output State |
|-----------|-------------|-----------------------|--------------|
| 0 | $$ \|0000\rangle$$ | 0 | $$ \|00000\rangle$$ |
| $\frac{1}{2}$ | $$ \|0001\rangle$$ | $\frac{1}{8}$ | $$0.9921567416 \|00010\rangle+\frac{1}{8} \|00011\rangle$$ |
| $\frac{1}{4}$ | $$ \|0010\rangle$$ | $\frac{1}{4}$ | $$\frac{\sqrt{15}}{4} \|00100\rangle+\frac{1}{4} \|00101\rangle$$ |
| $\frac{3}{4}$ | $$ \|0011\rangle$$ | $\frac{1}{12}$ | $$0.9965217286 \|00110\rangle+\frac{1}{12} \|00111\rangle$$ |
| $\frac{1}{8}$ | $$ \|0100\rangle$$ | $\frac{1}{2}$ | $$\frac{\sqrt{3}}{2} \|01000\rangle+\frac{1}{2} \|01001\rangle$$ |
| $\frac{5}{8}$ | $$ \|0101\rangle$$ | $\frac{1}{10}$ | $$0.9949874371 \|01010\rangle+\frac{1}{10} \|01011\rangle$$ |
| $\frac{3}{8}$ | $$ \|0110\rangle$$ | $\frac{1}{6}$ | $$0.9860132972 \|01100\rangle+\frac{1}{6} \|01101\rangle$$ |
| $\frac{7}{8}$ | $$ \|0111\rangle$$ | $\frac{1}{14}$ | $$0.9974457174 \|01110\rangle+\frac{1}{14} \|01111\rangle$$ |
| $\frac{1}{16}$ | $$ \|1000\rangle$$ | $1$ | $$ \|10001\rangle$$ |
| $\frac{9}{16}$ | $$ \|1001\rangle$$ | $\frac{1}{9}$ | $$0.99380799 \|10010\rangle+\frac{1}{9} \|10011\rangle$$ |
| $\frac{5}{16}$ | $$ \|1010\rangle$$ | $\frac{1}{5}$ | $$0.9797958971 \|10100\rangle+\frac{1}{5} \|10101\rangle$$ |
| $\frac{13}{16}$ | $$ \|1011\rangle$$ | $\frac{1}{13}$ | $$0.9970370305 \|10110\rangle+\frac{1}{13} \|10111\rangle$$ |
| $\frac{3}{16}$ | $$ \|1100\rangle$$ | $\frac{1}{3}$ | $$\frac{2 \sqrt{2}}{3} \|11000\rangle+\frac{1}{3} \|11001\rangle$$ |
| $\frac{11}{16}$ | $$ \|1101\rangle$$ | $\frac{1}{11}$ | $$0.9958591955 \|11010\rangle+\frac{1}{11} \|11011\rangle$$ |
| $\frac{7}{16}$ | $$ \|1110\rangle$$ | $\frac{1}{7}$ | $$0.9897433186 \|11100\rangle+\frac{1}{7} \|11101\rangle$$ |
| $\frac{15}{16}$ | $$ \|1111\rangle$$ | $\frac{1}{15}$ | $$0.9977753031 \|11110\rangle+\frac{1}{15} \|11111\rangle$$ |
